In [71]:
#!pip install plotly
#!pip install folium

In [101]:
import pandas as pd
import numpy as np

import pickle

import plotly.express as px

import folium

In [73]:
pkl_file = open('stations.pkl', 'rb')
df_stations = pickle.load(pkl_file)
pkl_file.close()

In [74]:
pkl_file = open('stopplaces.pkl', 'rb')
df_stopplaces = pickle.load(pkl_file)
pkl_file.close()

In [75]:
#df_stations = pd.read_csv('stations.csv')
#df_stopplaces = pd.read_csv('stopplaces.csv')

In [76]:
print(f'Stations: {df_stations.shape}')
print(f'Stopplaces: {df_stopplaces.shape}')

Stations: (5690, 16)
Stopplaces: (4208, 9)


In [77]:
df_stations.isna().sum()

id                      0
name                    0
metropolis              0
street                  8
houseNumber           893
postalCode              7
city                    4
state                   0
country                 0
stationCategory        12
owner                   0
organisationalUnit      0
countryCode             0
latitude              282
longitude             282
timeZone                0
dtype: int64

In [78]:
df_stopplaces.isna().sum()

id                       0
name                     0
availableTransports      0
transportAssociations    0
countryCode              0
state                    6
timeZone                 0
latitude                 0
longitude                0
dtype: int64

In [62]:
#missing_values = df_stations[df_stations.isna().any(axis=1)]
#print(missing_values)

In [79]:
px.bar(df_stations, x=['owner'], barmode='group')

In [80]:
df_stations.groupby(by='state').count()['id'].sort_values(ascending=False)

state
Bayern                    1025
Baden-Württemberg          720
Nordrhein-Westfalen        711
Hessen                     479
Sachsen                    478
Rheinland-Pfalz            419
Niedersachsen              357
Brandenburg                310
Sachsen-Anhalt             289
Thüringen                  289
Mecklenburg-Vorpommern     180
Schleswig-Holstein         137
Berlin                     133
Saarland                    77
Hamburg                     58
Bremen                      16
Schweiz CH                  12
Name: id, dtype: int64

## Join Stations and Stop Places

In [81]:
df_stopplaces.drop(columns=['name', 'state', 'countryCode', 'latitude', 'longitude','timeZone'], inplace=True)

In [86]:
df_stations['id'] = df_stations['id'].astype(int)
df_stations['latitude'] = df_stations['latitude'].astype(float)
df_stations['longitude'] = df_stations['longitude'].astype(float)

df_stations.dtypes

id                      int32
name                   object
metropolis             object
street                 object
houseNumber            object
postalCode             object
city                   object
state                  object
country                object
stationCategory        object
owner                  object
organisationalUnit     object
countryCode            object
latitude              float64
longitude             float64
timeZone               object
dtype: object

In [88]:
df_stopplaces['id'] = df_stopplaces['id'].astype(int)
df_stopplaces.dtypes

id                        int32
availableTransports      object
transportAssociations    object
dtype: object

In [89]:
df = df_stations.join(df_stopplaces, on='id', how='left', rsuffix='_s')
df.drop(columns=['id_s'], inplace=True)

In [94]:
df['availableTransports']

0                        [REGIONAL_TRAIN]
1       [CITY_TRAIN, BUS, REGIONAL_TRAIN]
2                        [REGIONAL_TRAIN]
3                   [BUS, REGIONAL_TRAIN]
4                        [REGIONAL_TRAIN]
                      ...                
5685                     [REGIONAL_TRAIN]
5686                     [REGIONAL_TRAIN]
5687                     [REGIONAL_TRAIN]
5688                [BUS, REGIONAL_TRAIN]
5689                [BUS, REGIONAL_TRAIN]
Name: availableTransports, Length: 5690, dtype: object

In [111]:
transports = []
for entry in df['transportAssociations']:
    try:
        for e in entry:
            transports.append(e)
    except:
        pass

transportAssociations = pd.Series(transports).value_counts()

In [112]:
transports = []
for entry in df['availableTransports']:
    try:
        for e in entry:
            transports.append(e)
    except:
        pass

availableTransports = pd.Series(transports).value_counts()

In [113]:
px.bar(transportAssociations)

In [114]:
px.bar(availableTransports)

## Display map

In [115]:
df_stations.dropna(subset = ['latitude'], inplace=True)

In [116]:
df_bw = df_stations[df_stations['state']=='Baden-Württemberg']

In [117]:
len(df_bw)

691

In [120]:
m = folium.Map(location=[50.111, 8.682],zoom_start=6)
for i in df_bw.index:
    folium.Marker(location=[ df_bw['latitude'][i], df_bw['longitude'][i] ], fill_color='#43d9de', radius=8,tooltip=df_bw['name'][i]).add_to(m)

#m